In [2]:
%pip install -q google-genai
%pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 14.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [3]:
# Google Gemini API Key configuration
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [4]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [5]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [6]:
##########################################
# Agent A: Agenda Decider                #
##########################################
def agenda_decider_agent(date):

    agenda_decider = Agent(
        name="agenda_decider",
        model="gemini-2.0-flash", # Cost-effective choice for this agent (quick, simple task involved, etc.)
        instruction="""
        The Agenda Decider Agent initiates a bedtime daily story creation by first checking for globally significant events which happen(ed) on the given date.
        If found, it prioritizes one to set the story's country and context - in this case, the story must be related to the prioritized event.
        Otherwise, it randomly selects a country to focus on. It also randomly chooses
        a positive human value to compose the story. Finally, it generates a concise agenda for the Writer Agent as a simple string,
        including the focus country, core human value, prioritized event (if one was selected), and optional context containing cultural/story starter suggestions, separated by semicolons.

        Example output:
        Country: Japan; Human Value: Kindness; Event: Cherry Blossom festival; Context: In a small village nestled among cherry trees, two children, Hana and Kenji, discover the true meaning of kindness during the beautiful spring festival.
        """,
        description="Agent which decides the agenda for the daily bedtime story",
        tools=[google_search]
    )

    agenda_decider_input = f"Date: {date}"
    agenda_string = call_agent(agenda_decider, agenda_decider_input)
    return agenda_string

In [16]:
##########################################
# Agent B: Writer                        #
##########################################
def writer_agent(agenda_string):

    writer = Agent(
        name="writer_agent",
        model="gemini-2.0-flash",
        instruction=f"""
        You are a skilled children's story writer. Based on the agenda provided as a string, write a short (approximately 3 minutes reading time for an adult) and engaging bedtime story for children aged 5-10.

        The agenda string contains the following information, separated by commas:
        - Focus Country
        - Core Human Value
        - (Optional) Prioritized Event
        - (Optional) Cultural Context Snippet / Story Starter Suggestion

        Please extract these pieces of information from the string to guide your story. If a piece of information is not provided, use your creativity to fill in the details.

        The story should:
        - Be set in the focus country.
        - Explore the human value.
        - (If available) Incorporate the cultural context.
        - (If available) Consider the story starter.
        - Feature child characters, ideally reflecting the cultural context of the story's setting.
        - Have a clear beginning, middle, and end.
        - Convey a positive message related to the core human value in an age-appropriate manner.
        - Be written in simple and accessible language suitable for reading aloud.
        - Aim to be entertaining and capture the imagination of young children.

        Do not include any illustrations or formatting other than the story text itself.

        Write the story now based on the following agenda:
        {agenda_string}
        """,
        description="Agent responsible for writing the daily bedtime story based on the provided agenda string."
    )

    story = call_agent(writer, agenda_string)
    return story

In [20]:
##########################################
# --- Agent C: Scene Descriptor      --- #
##########################################
def scene_descriptor_agent(story):

    scene_descriptor = Agent(
        name="scene_descriptor_agent",
        model="gemini-2.0-flash",
        instruction=f"""
        You are a visual scene describer for a children's story. Based on the story given, generate detailed textual descriptions of six key scenes.
        Each description should correspond to approximately 30 seconds of reading time within the 3-minute story, focusing on the main visual elements and actions occurring in that segment.
        For each scene, clearly describe the characters involved, the setting, their actions, and any significant
        objects or atmosphere. These descriptions will be used by an AI model to generate illustrations.
        The output should be a string containing six scene descriptions. Each description should be a concise and vivid paragraph.
        Each of the scenes must be separated from the other by "\n ------- SCENE ------- \n"
        Story: {story}
        """,
        description="Agent responsible for describing the story scenes for illustration."
    )

    scene_descriptions = call_agent(scene_descriptor, story)
    return scene_descriptions


In [9]:
project_description_en = '''
✨ Gemini's Sleepytime Tales - For children all over the world 👧👦🌎
🌙 A world of stories to dream about ✨
📖🎧 Multicultural 3-minute stories to read and listen to at bedtime. 🌱 Promoting values and the joy of learning together! 🌈📚
'''
project_description_pt = '''
✨ Historinhas do Gemini - Para crianças do mundo todo 👧👦🌎
🌙 Um mundo de histórias para sonhar ✨
📖🎧 Histórias multiculturais de 3 minutos para ler e ouvir na hora de dormir. 🌱 Promovendo valores e a alegria de aprender juntos! 🌈📚
'''
print(project_description_en)
current_date = date.today().strftime("%d/%m/%Y")


✨ Gemini's Sleepytime Tales - For children all over the world 👧👦🌎
🌙 A world of stories to dream about ✨
📖🎧 Multicultural 3-minute stories to read and listen to at bedtime. 🌱 Promoting values and the joy of learning together! 🌈📚



In [10]:
story_agenda = agenda_decider_agent(current_date)
print("\n--- 📝 Agent 1 result ---\n")
display(to_markdown(story_agenda))
print("--------------------------------------------------------------")


--- 📝 Agent 1 result ---



> Okay, I will create a bedtime story agenda for May 17, 2025. I will first check for globally significant events that occurred on this date. If there is a significant event, I will prioritize it to set the story's country and context. Otherwise, I will randomly select a country. I will also randomly choose a positive human value. Finally, I will generate a concise agenda for the Writer Agent, including the focus country, core human value, prioritized event (if one was selected), and optional context containing cultural/story starter suggestions, separated by semicolons.
> 
> 
> Okay, after reviewing the search results, here's an agenda for the bedtime story:
> 
> **Country:** Norway; **Human Value:** Perseverance; **Event:** Constitution Day of Norway (May 17th); **Context:** In a small Norwegian village, a young girl named Astrid is determined to participate in the Constitution Day parade, despite facing a challenge that tests her resolve and the support of her community. Perhaps she has an injury or a problem with her traditional costume that needs to be overcome.
> 


--------------------------------------------------------------


In [18]:
story = writer_agent(story_agenda)
print("\n--- 📝 Agent 2 result ---\n")
display(to_markdown(story))
print("--------------------------------------------------------------")


--- 📝 Agent 2 result ---



> In a cozy little village nestled among the fjords of Norway, lived a girl named Astrid. Astrid loved Constitution Day, May 17th! It was a day of flags, parades, and delicious waffles. Every year, she marched proudly in the children's parade, wearing her beautiful bunad, a traditional Norwegian costume.
> 
> This year, Astrid was especially excited. She had practiced twirling her little Norwegian flag perfectly. But the day before the parade, disaster struck! While playing hide-and-seek with her friends, Astrid tripped and tore a big hole in the skirt of her bunad.
> 
> Astrid's heart sank. The bunad had been passed down from her grandmother, and it was very special. Tears welled up in her eyes. "I can't march in the parade now!" she cried.
> 
> Her mother hugged her tight. "Don't worry, Astrid," she said. "We'll figure something out. It won't be easy, but we will try."
> 
> They tried to sew the tear, but the fabric was old and delicate. The stitches kept pulling through. Astrid felt like giving up. "It's no use, Mama," she said sadly. "I guess I'll just have to watch the parade from the sidelines."
> 
> But Mama smiled. "Astrid," she said, "Remember what Grandma always said? Norwegians are known for their 'sisu' - their perseverance! We don't give up easily."
> 
> Inspired, Astrid decided to try a different approach. She remembered seeing her grandfather patching fishing nets. Maybe she could patch the bunad too! With Mama's help, Astrid carefully cut a small piece of matching fabric from an old scarf. Slowly and carefully, she began to weave the patch into the torn fabric.
> 
> It was slow work, and her fingers ached. But Astrid kept going, remembering her grandmother and the spirit of perseverance. Finally, just as the sun began to rise on May 17th, the bunad was mended! The patch wasn't perfect, but it held strong.
> 
> Astrid beamed with pride. She put on her bunad, grabbed her flag, and ran to join the parade. As she marched through the village, singing Norwegian songs with her friends, she felt a sense of accomplishment she had never felt before. She had faced a challenge, worked hard, and persevered.
> 
> The parade was wonderful. And Astrid knew, as she ate a delicious waffle afterwards, that this Constitution Day was the best one yet. Not because of the flags or the waffles, but because she had learned that even when things get tough, with a little perseverance, you can overcome anything. And that’s a very Norwegian thing to do. Now close your eyes and remember Astrid, and know that you too can persevere. Good night!
> 


--------------------------------------------------------------


In [19]:
scenes = scene_descriptor_agent(story)
print("\n--- 📝 Agent 3 result ---\n")
print(scenes)
print("--------------------------------------------------------------")


--- 📝 Agent 3 result ---

A young girl named Astrid stands in a picturesque Norwegian village, characterized by colorful houses and towering fjords in the background. She's beaming with excitement, dressed in a beautiful, traditional bunad costume, and holding a small Norwegian flag. Astrid is practicing twirling the flag with perfect precision, her eyes sparkling with anticipation for Constitution Day.

 ------- SCENE ------- 

Astrid, still in her bunad, is playing hide-and-seek with two friends in a grassy field near the village. She is in the middle of tripping over a root, her flag falling from her grasp. A large tear is visible in the skirt of her bunad as she falls, her expression changing from joy to alarm. Her friends look on with concern.

 ------- SCENE ------- 

Close-up on Astrid, sitting on a stool in a cozy room. Tears stream down her face as she stares at the torn bunad in her lap. The hole in the skirt is clearly visible. Her mother kneels beside her, offering a comfo